<a href="https://colab.research.google.com/github/sarahgruetz/AluraChallenge_DS_2/blob/main/Alura_Challenge_semanas_3_e_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 38.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3d5d947076dce73f0ddbef2757c3ebd967b7253b7c280f7ab323b100ac664b3b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


### Iniciando a SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Alura Challenge") \
    .getOrCreate()

In [3]:
import zipfile

zipfile.ZipFile('/content/drive/MyDrive/curso-spark/ChallengeAlura/semanas-3-e-4.zip','r')\
        .extractall('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados')

In [21]:
dataset = spark.read.parquet('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados/dataset_ml_parquet')

In [22]:
print(f'O dataset possui {dataset.count()} registros e {len(dataset.columns)} colunas.')

O dataset possui 66551 registros e 25 colunas.


In [5]:
dataset.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

In [6]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- Zona Central: integer (nullable = true)
 |-- Zona Norte: integer (nullable = true)
 |-- Zona Oeste: integer (nullable = true)
 |-- Zona Sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- Animais permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- Condomínio fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- Portaria 24h: integer (nullable = true)
 |-- Portão eletrônico: integer (nullable 

In [12]:
from pyspark.sql import functions as f

In [13]:
dataset\
  .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
  .show(vertical=True)

-RECORD 0-----------------
 id                 | 0   
 andar              | 0   
 area_util          | 0   
 banheiros          | 0   
 quartos            | 0   
 suites             | 0   
 vaga               | 0   
 bairro             | 0   
 condominio         | 0   
 iptu               | 0   
 valor              | 0   
 Zona Central       | 0   
 Zona Norte         | 0   
 Zona Oeste         | 0   
 Zona Sul           | 0   
 Academia           | 0   
 Animais permitidos | 0   
 Churrasqueira      | 0   
 Condomínio fechado | 0   
 Elevador           | 0   
 Piscina            | 0   
 Playground         | 0   
 Portaria 24h       | 0   
 Portão eletrônico  | 0   
 Salão de festas    | 0   



### Preparando os dados

In [7]:
from pyspark.ml.feature import VectorAssembler

In [23]:
dataset.columns

['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [25]:
X = [
    'andar',
    'area_util',
    'banheiros',
    'quartos',
    'suites',
    'vaga',
    'condominio',
    'iptu',
    'valor',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul',
    'Academia',
    'Animais permitidos',
    'Churrasqueira',
    'Condomínio fechado',
    'Elevador',
    'Piscina',
    'Playground',
    'Portaria 24h',
    'Portão eletrônico',
    'Salão de festas'
]

In [26]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [27]:
dataset_modelo = assembler.transform(dataset).select('features','id')

In [28]:
dataset_modelo.show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------+------------------------------------+
|features                                                                                                    |id                                  |
+------------------------------------------------------------------------------------------------------------+------------------------------------+
|[2.0,35.0,1.0,1.0,0.0,0.0,100.0,100.0,245000.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]     |00002dd9-cc74-4809-b5a5-850adf0e7526|
|(23,[0,1,2,3,5,6,7,8,10,15,17,19,20,22],[1.0,84.0,2.0,2.0,1.0,770.0,105.0,474980.0,1.0,1.0,1.0,1.0,1.0,1.0])|0009ca94-2b37-4381-b8b8-773ce0f92444|
|(23,[1,2,3,6,7,8,12,14,17],[85.0,2.0,2.0,460.0,661.0,290000.0,1.0,1.0,1.0])                                 |000e3d28-e3e5-4110-b488-69154931140e|
|(23,[1,2,3,5,6,7,8,11,18,19],[58.0,1.0,2.0,1.0,550.0,550.0,249000.0,1.0,1.0,1.0])                           |00

### Padronização dos dados

Para realizar a clusterização dos dados é preciso que as variáveis tenham valores mais próximos uma das outras. Uma forma de se obter isso é utilizando o StandardScaler que realiza a padronização dos dados subtraindo cada coluna por sua média e dividindo pelo seu desvio padrão. Dessa forma, a distribuição de todas as variáveis terá uma média zero e desvio padrão igual a um.

In [29]:
from pyspark.ml.feature import StandardScaler

In [30]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(dataset_modelo)
dataset_modelo_scaler = scaler_model.transform(dataset_modelo)   # fit_transform (?)

In [31]:
dataset_modelo_scaler.show(5)

+--------------------+--------------------+--------------------+
|            features|                  id|     scaled_features|
+--------------------+--------------------+--------------------+
|[2.0,35.0,1.0,1.0...|00002dd9-cc74-480...|[0.13607726247524...|
|(23,[0,1,2,3,5,6,...|0009ca94-2b37-438...|(23,[0,1,2,3,5,6,...|
|(23,[1,2,3,6,7,8,...|000e3d28-e3e5-411...|(23,[1,2,3,6,7,8,...|
|(23,[1,2,3,5,6,7,...|000fb707-6cad-496...|(23,[1,2,3,5,6,7,...|
|(23,[1,2,3,4,5,6,...|001b6db0-e88d-4eb...|(23,[1,2,3,4,5,6,...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



### Redução de dimensionalidade

Para redução de dimensionalidade foi utilizado o PCA. Essa redução é necessária para permitir a visualização dos dados

In [32]:
from pyspark.ml.feature import PCA

In [33]:
pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dataset_modelo_scaler)
dataset_modelo_pca = model_pca.transform(dataset_modelo_scaler)

In [36]:
dataset_modelo_pca.select('pca_features').show(5, False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-6.165125049533812,1.3380985269405694]  |
|[-3.2529111812184865,-1.1179591836228857]|
|[-1.0611769329629537,-1.6685040058694294]|
|[-1.995811900783872,-1.7655796610847843] |
|[-0.4181406070145923,-2.131040791141693] |
+-----------------------------------------+
only showing top 5 rows



### Criando um pipeline

In [37]:
from pyspark.ml import Pipeline

In [38]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='scaled_features'),
                                PCA(k=2, inputCol='scaled_features', outputCol='pca_features')])

In [39]:
pca_pipeline_model = pca_pipeline.fit(dataset)

In [40]:
dados_pipeline_model = pca_pipeline_model.transform(dataset)

In [44]:
dados_pipeline_model.select('pca_features').show(5, False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-6.165125049533812,1.3380985269405694]  |
|[-3.2529111812184865,-1.1179591836228857]|
|[-1.0611769329629537,-1.6685040058694294]|
|[-1.995811900783872,-1.7655796610847843] |
|[-0.4181406070145923,-2.131040791141693] |
+-----------------------------------------+
only showing top 5 rows



### Criando os clusters

In [42]:
from pyspark.ml.clustering import KMeans

In [43]:
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca').setK(5)

In [45]:
model_kmeans = kmeans.fit(dados_pipeline_model)

In [46]:
prections_kmeans = model_kmeans.transform(dados_pipeline_model)

In [47]:
prections_kmeans.select('id', 'cluster_pca').show(5,False)

+------------------------------------+-----------+
|id                                  |cluster_pca|
+------------------------------------+-----------+
|00002dd9-cc74-4809-b5a5-850adf0e7526|1          |
|0009ca94-2b37-4381-b8b8-773ce0f92444|3          |
|000e3d28-e3e5-4110-b488-69154931140e|0          |
|000fb707-6cad-496d-8cb7-d8046cb5ef37|3          |
|001b6db0-e88d-4eba-84e5-0ef94b091a64|0          |
+------------------------------------+-----------+
only showing top 5 rows



### Filtrando imoveis do mesmo cluster

In [48]:
cluster_imoveis = prections_kmeans.select('id', 'cluster_pca')

In [49]:
dataset = dataset\
              .join(cluster_imoveis,'id', how='inner')

In [50]:
dataset.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-----------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|cluster_pca|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-----------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|24

In [62]:
dataset.groupBy('cluster_pca').count().show()

+-----------+-----+
|cluster_pca|count|
+-----------+-----+
|          1|19298|
|          3|15707|
|          4| 6983|
|          2| 6092|
|          0|18471|
+-----------+-----+



In [60]:
cluster_means = dataset.groupBy('cluster_pca').mean()
cluster_means.show()

+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+-------------------+-------------------+-------------------+--------------------+-----------------------+--------------------+-----------------------+-------------------+-------------------+-------------------+-------------------+----------------------+--------------------+----------------+
|cluster_pca|        avg(andar)|    avg(area_util)|    avg(banheiros)|      avg(quartos)|       avg(suites)|         avg(vaga)|   avg(condominio)|         avg(iptu)|       avg(valor)|   avg(Zona Central)|    avg(Zona Norte)|    avg(Zona Oeste)|      avg(Zona Sul)|       avg(Academia)|avg(Animais permitidos)|  avg(Churrasqueira)|avg(Condomínio fechado)|      avg(Elevador)|       avg(Piscina)|    avg(Playground)|  avg(Portaria 24h)|avg(Portão eletrônico)|avg(Salão de festas)|avg(cluster_pca)|
+-------